在创建张量的时候设置一个参数`requires_grad=True`，意味着这个张量会加入到计算图中，作为计算图的叶子节点参与计算。
每个张量都有一个grad_fn方法，这个方法包含着创建该张量的运算的导数信息。在反向传播过程中，通过传入后一层的神经网络的梯度，该函数会计算出参与计算的所有张量的梯度。grad_fn本身有一个next_functions属性，包含连接该张量的其他张量grad_fn。通过不断反向传播回溯中间张量的计算节点，可以得到所有张量的梯度。一个张量的梯度张量的信息保存在该张量的grad属性中。

此外还有一个专门求导的包，即`torch.autograd`，它有两个重要的函数`torch.autograd.backward`和`torch.autograd.grad`。torch.autograd.backward函数通过传入根结点张量，以及初始梯度张量，可以计算产生该根结点所有对应的叶子节点的梯度。如果要在反向传播的时候保留计算图，可以设置`retain_graph=True`。如果需要创建方向计算图，可以设置`create_graph=True`。

In [2]:
import torch
t1 = torch.randn(3, 3, requires_grad=True)

In [3]:
t1

tensor([[ 1.4973, -0.7848, -0.7087],
        [ 1.8447,  0.4992, -0.4637],
        [ 0.9885, -0.8224, -0.1643]], requires_grad=True)

In [4]:
t2 = t1.pow(2).sum() # 计算张量的所有分量平方和

In [5]:
t2.backward() # 反向传播

In [6]:
t1.grad 

tensor([[ 2.9945, -1.5696, -1.4173],
        [ 3.6894,  0.9983, -0.9274],
        [ 1.9770, -1.6447, -0.3286]])

In [7]:
t1.grad.zero_() # 单个张量清零梯度的方法

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

有时候，不需要求出当前张量对所有产生该张量的叶子节点的梯度名，这是可以使用torch.autograd.grad函数。这个函数的参数是两个张量，第一个张量是计算图的数据结果张量，第二个张量是需要对计算图求导的张量。最后输出的结果是第一个张量对第二个张量求导的结果

In [8]:
t1 = torch.randn(3, 3, requires_grad=True)

In [9]:
t2 = t1.pow(2).sum()

In [10]:
torch.autograd.grad(t2, t1)

(tensor([[ 0.0481, -3.4426,  0.9009],
         [-0.8894,  0.0872, -1.7452],
         [ 2.0138, -1.4716, -1.6325]]),)

# 计算图构建的启用和禁用

由于计算图的构建需要消耗内存和计算资源，在一些情况下，计算图并不是必要的，可以在`torch.no_grad`上下文管理器中就不会构建。

另外，还有一个情况是对于一个张量，我们在反向传播的时候可能不需要让梯度通过这个张量的节点，也就是新建的计算图要和原来的计算图分离。这种情况下，可以使用张量的detach方法，通过调用这个方法，可以返回一个新的张量，该张量会成为一个新的计算图的叶子节点，新的计算图和老得计算图相互分离，互不影响。

In [12]:
t1 = torch.randn(3, 3, requires_grad=True)

In [15]:
t2 = t1.sum()

In [16]:
t2

tensor(0.7787, grad_fn=<SumBackward0>)

In [17]:
with torch.no_grad():
    t3 = t1.sum()

In [18]:
t3

tensor(0.7787)

In [19]:
t1.sum().detach() # 和原来的计算图分离

tensor(0.7787)